In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras import datasets # https://keras.io/api/datasets/
from tensorflow.keras import preprocessing # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing
from tensorflow.keras import layers # https://keras.io/api/layers/
from tensorflow.keras import activations # https://keras.io/api/layers/activations/
from tensorflow.keras import losses # https://keras.io/api/losses/
from tensorflow.keras import metrics # https://keras.io/api/metrics/
from tensorflow.keras import optimizers # https://keras.io/api/optimizers/
from tensorflow.keras import regularizers # https://www.tensorflow.org/api_docs/python/tf/keras/regularizers

In [2]:
# for ImageNet dataset
num_classes = 1000
input_shape = (299, 299, 3)

In [3]:
# actual inception model
# core = tf.keras.applications.InceptionV3()
# core.summary()

In [4]:
# example of how to build the build a CNN network that has an inception block 
# code adapted from: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/keras/applications/inception_v3.py

# use sequential model
model = tf.keras.Sequential()

# specify input shape (H, W, C)
inputs = tf.keras.Input(shape=(299, 299, 3))

# regular sequence of conv2d... maxpool
x = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2), padding='valid')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

# regular sequence of conv2d... maxpool
x = layers.Conv2D(filters=80, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

# inception block
branch1x1 = layers.Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
branch1x1 = layers.BatchNormalization()(branch1x1)
branch1x1 = layers.Activation('relu')(branch1x1)

branch5x5 = layers.Conv2D(filters=48, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
branch5x5 = layers.BatchNormalization()(branch5x5)
branch5x5 = layers.Activation('relu')(branch5x5)
branch5x5 = layers.Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), padding='same')(branch5x5)
branch5x5 = layers.BatchNormalization()(branch5x5)
branch5x5 = layers.Activation('relu')(branch5x5)

branch3x3dbl =  layers.Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
branch3x3dbl = layers.BatchNormalization()(branch3x3dbl)
branch3x3dbl = layers.Activation('relu')(branch3x3dbl)
branch3x3dbl = layers.Conv2D(filters=96, kernel_size=(3, 3), strides=(1, 1), padding='same')(branch3x3dbl)
branch3x3dbl = layers.BatchNormalization()(branch3x3dbl)
branch3x3dbl = layers.Activation('relu')(branch3x3dbl)
branch3x3dbl =  layers.Conv2D(filters=96, kernel_size=(3, 3), strides=(1, 1), padding='same')(branch3x3dbl)
branch3x3dbl = layers.BatchNormalization()(branch3x3dbl)
branch3x3dbl = layers.Activation('relu')(branch3x3dbl)

branch_pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch_pool = layers.Conv2D(filters=32, kernel_size=(1, 1), strides=(1, 1), padding='same')(branch_pool)
branch_pool = layers.BatchNormalization()(branch_pool)
branch_pool = layers.Activation('relu')(branch_pool)

x = layers.concatenate(name='inception_block', inputs=[branch1x1, branch5x5, branch3x3dbl, branch_pool])

# classification block
x = layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
outputs = layers.Dense(name='predictions', units=1000, activation='softmax')(x)

# model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________